In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data_ozon.csv', parse_dates=['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254748 entries, 0 to 254747
Data columns (total 17 columns):
 #   Column                                                 Non-Null Count   Dtype         
---  ------                                                 --------------   -----         
 0   Ozon ID                                                254748 non-null  int64         
 1   Уникальные посетители, всего                           254748 non-null  int64         
 2   Уникальные посетители с просмотром карточки товара     254748 non-null  int64         
 3   Показы, всего                                          254748 non-null  int64         
 4   Показы на карточке товара                              254748 non-null  int64         
 5   Показы в поиске и каталоге                             254748 non-null  int64         
 6   В корзину, всего                                       254748 non-null  int64         
 7   Заказано товаров                                       2

In [ ]:
# Воронка продаж
ls_param = [
    'Возвращено товаров', 'Отменено товаров', 'Доставлено (за выбранный период)',
    'Заказано товаров', 'В корзину, всего',
    'Уникальные посетители с просмотром карточки товара',
    'Показы на карточке товара', 'Показы в поиске и каталоге',
    'Уникальные посетители, всего', 'Показы, всего']
df[ls_param].sum(axis=0)

Возвращено товаров                                         145
Отменено товаров                                           481
Доставлено (за выбранный период)                          8539
Заказано товаров                                          9427
В корзину, всего                                         54603
Уникальные посетители с просмотром карточки товара      474132
Показы на карточке товара                               744891
Показы в поиске и каталоге                             7328221
Уникальные посетители, всего                          12647514
Показы, всего                                         22389605
dtype: int64

* Имеющиеся данные образуют воронку продаж, следовательно есть взамосвязь, у выбранных параметров.
* Целевым параметром выберем "Заказано товаров", все что ниже могут стать входящими.
* Сгруппируем по Ozon ID

In [ ]:
ls_param = [
    'Ozon ID',  'Показы, всего', 'Уникальные посетители, всего',
    'Показы в поиске и каталоге', 'Показы на карточке товара',
    'Уникальные посетители с просмотром карточки товара', 'В корзину, всего',
    'Заказано товаров']
df1 = df[ls_param].groupby('Ozon ID').sum()
df1[df1['Заказано товаров'] > 0].head()

,"Показы, всего","Уникальные посетители, всего",Показы в поиске и каталоге,Показы на карточке товара,Уникальные посетители с просмотром карточки товара,"В корзину, всего",Заказано товаров
Ozon ID,,,,,,,
260912035,309,202,95,34,21,4,2
260919692,360133,205340,191617,6908,4577,452,157
275493676,269938,111827,61675,10020,5554,334,19
275500058,116307,58179,40371,3783,2429,161,6
276736894,3607,1765,1634,151,69,8,3


По товарам также очевидна воронка (слева цифры больше)

In [ ]:
df2 = df1[df1['Заказано товаров'] > 0]
y = df2['Заказано товаров']
x = df2.drop(columns=['Заказано товаров'])
bestfeatures = SelectKBest(score_func=chi2, k='all')
fit = bestfeatures.fit(x, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x.columns)
featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns = ['Параметр', 'Score']
featureScores.nlargest(10, 'Score')

,Параметр,Score
0,"Показы, всего",1.221507e+08
1,"Уникальные посетители, всего",4.886271e+07
2,Показы в поиске и каталоге,1.835321e+07
3,Показы на карточке товара,7.990527e+06
4,Уникальные посетители с просмотром карточки то...,4.386308e+06
5,"В корзину, всего",3.527533e+05


Методы машинного обучения также подтверждают воронку продаж (вверху цифры больше)